In [44]:
from __future__ import unicode_literals
from numpy.core.numeric import NaN
from regex.regex import escape
from collections import Counter
import regex, copy, heapq, math, operator, random
import pandas as pd
import numpy as np
import sys,os,sys

In [45]:
# pip install hazm
from hazm import *

## Reading data

In [46]:
docs_links = pd.read_csv('IR_Spring2021_ph12_7k.csv')
docs_links = docs_links[docs_links['content'].notnull()]

documents = pd.read_csv('IR_Spring2021_ph12_7k.csv').iloc[:,1].values.tolist()
links = pd.read_csv('IR_Spring2021_ph12_7k.csv').iloc[:,2].values.tolist()

punctuations = pd.read_csv("punctuations.csv")
punctuations_to_remove = ''.join(punctuations["punctuations"].values.tolist()).replace(" ","")
stopwords_to_remove = ''.join(stopwords_list()).replace(" ","")

## Tokenization

In [47]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
N=len(docs_links["content"])
docs_len = {}

def tokenize(id, document):
    document = str(document)
    document = document.replace("انتهای پیام","")
    n_document = str(normalizer.normalize(document))
    #remove punctuations
    n_document = n_document.translate(str.maketrans('','',punctuations_to_remove))
    #remove stop words
    tokens = list(map(lambda t:lemmatizer.lemmatize(t),word_tokenize(n_document))) 
    dictionary = dict(Counter(tokens))
    for k, tf in dictionary.items():
        dictionary[k] = 1 + math.log10(tf)
    if id is not None:
        docs_len[id] = dictionary
    return dictionary

## Build Weighted Inverted Index

In [48]:
def build_weighted_inv_ind(docs_links):
    dictionary = {}
    for id, doc in zip(docs_links["id"], docs_links["content"]):
        counts = tokenize(id, doc)
        for k,v in counts.items():
            if len(k) < 3:
                continue
            if k in dictionary.keys():
                dictionary[k][0] += 1
                dictionary[k][1][id] = v
            else:
                dictionary[k]=[1,{id:v}]
    # calc idfs 
    for k in dictionary.keys():
        dictionary[k][0] = math.log10(N/dictionary[k][0])
    
    for id, dl in docs_len.items():
        size_d = 0
        for t, w in dl.items():
            if t in dictionary:
                size_d += dictionary[t][0]*w
        docs_len[id] = math.sqrt(size_d)
    return dictionary


## Build Champion Lists

In [49]:
def build_champion_lists(weighted_inv_ind):
    champion_lists = {}
    for t, ws in weighted_inv_ind.items():
        champion_lists[t] = []
        champion_lists[t].append(ws[0])
        champion_lists[t].append(dict(sorted(ws[1].items(), key=operator.itemgetter(1), reverse=True)[:10]))
    return champion_lists

In [50]:
%%time
weighted_inv_ind = build_weighted_inv_ind(docs_links)
champion_lists = build_champion_lists(weighted_inv_ind)

CPU times: user 5.82 s, sys: 57.3 ms, total: 5.87 s
Wall time: 5.88 s


## Compute Similarity

In [51]:
N=len(docs_links["content"])
def compute_similarity(nd_q, weighted_inv_ind):
    similarities = {}
    for k,v in nd_q.items():
        if k not in weighted_inv_ind:
            continue
        #idf = math.log(N/weighted_inv_ind[k][0])
        idf = weighted_inv_ind[k][0]
        for docid, w in weighted_inv_ind[k][1].items():
            if docid in similarities.keys():
                similarities[docid] += idf*w*(1+math.log10(v))
            else:
                similarities[docid] = idf*w*v
    for doc_id, sim in similarities.items():
        similarities[doc_id] = sim/docs_len[doc_id]
    return similarities

## Get the best scores by calculating the most similar indexes to the query

In [52]:
def get_k_most_sim(query, weighted_inv_ind, use_heap, k=5):
    scores = compute_similarity(tokenize(None, query), weighted_inv_ind)
    best_scores =[]
    heap = []
    if use_heap == True:
        for docid,score in scores.items():
            heapq.heappush(heap,(-score, docid))
        for _ in range(k):
            best_scores.append(heapq.heappop(heap))
    else:
        best_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse = True)}
        best_scores = list(best_scores)[:k]
    return best_scores

In [54]:
def get_links(top_k, docs_links, use_heap):
    results=[]
    if use_heap == True:
        for i in top_k:
            results.append(docs_links.loc[docs_links.index[docs_links['id'] == i[1]].tolist()[0]]["url"])
    else:
        for i in top_k:
            results.append(docs_links.loc[docs_links.index[docs_links['id'] == i].tolist()[0]]["url"])
    return results

## A few examples:

In [57]:
top_k = get_k_most_sim("واکسیناسیون بیماری کرونا در جهان", weighted_inv_ind, True, 6)
top_k

[(-0.469636181566958, 6458),
 (-0.469636181566958, 6464),
 (-0.45463906624891315, 6408),
 (-0.4460984032732113, 5964),
 (-0.40550846405575436, 6005),
 (-0.38370132008348123, 149)]

In [58]:
get_links(top_k, docs_links, True)

['https://www.isna.ir/news/99120705273/آغاز-واکسیناسیون-سالمندان-و-جانبازان-آسایشگاه-ها-علیه-کووید-۱۹-عکس',
 'https://www.isna.ir/news/99120705273/آغاز-واکسیناسیون-سالمندان-و-جانبازان-آسایشگاه-ها-علیه-کووید-۱۹-عکس',
 'https://www.isna.ir/news/99112014708/واکسن-کرونا-کی-به-استان-مرکزی-می-رسد',
 'https://www.isna.ir/news/99061510873/کاهش-پوشش-واکسیناسیون-کودکان-در-کشور-تحت-تاثیر-کرونا',
 'https://www.isna.ir/news/99062519519/آیا-موج-سوم-کرونا-در-کشور-آغاز-می-شود-یا-خیر',
 'https://www.isna.ir/news/99030804393/قهرمان-کشتی-المپیک-کرونا-گرفت']

In [59]:
top_k = get_k_most_sim("وام‌های اشتغال‌زایی عشایری و روستایی", weighted_inv_ind, True, 6)
get_links(top_k, docs_links, True)

['https://www.isna.ir/news/98070201774/بررسی-طرح-اصلاح-قانون-پرداخت-تسهیلات-اشتغال-زایی-روستائیان-در',
 'https://www.isna.ir/news/98070201774/بررسی-طرح-اصلاح-قانون-پرداخت-تسهیلات-اشتغال-زایی-روستائیان-در',
 'https://www.isna.ir/news/98070907368/تعویق-سه-ساله-بازگشت-تسهیلات-اشتغال-روستایی-و-عشایری',
 'https://www.isna.ir/news/98041005289/صندوق-کارآفرینی-امید-امسال-۱۰۰-هزار-شغل-ایجاد-می-کند',
 'https://www.isna.ir/news/98031807669/کندی-آهنگ-پرداخت-تسهیلات-از-سوی-بانک-های-کرمانشاه',
 'https://www.isna.ir/news/98050502291/حذف-یارانه-دهک-های-پردرآمد-کلید-خورد']

In [60]:
top_k = get_k_most_sim("پرونده الکترونیک سلامت", weighted_inv_ind, True, 6)
get_links(top_k, docs_links, True)

['https://www.isna.ir/news/98020201014/اعضای-کمیسیون-برنامه-از-سامانه-سنا-بازدید-کردند',
 'https://www.isna.ir/news/98121008034/آغاز-ثبت-اعتراض-حذف-شدگان-مسکن-ملی-از-امروز',
 'https://www.isna.ir/news/98081911066/جزئیات-برخورداری-از-یارانه-دولتی-بیمه-تکلیف-بیمه-مردم-باید-روشن',
 'https://www.isna.ir/news/99121108369/هزینه-۱۱۰-میلیاردی-بستری-و-درمان-بیماران-کرونا-در-بوشهر',
 'https://www.isna.ir/news/99091007815/گزارش-رئیس-سازمان-بیمه-سلامت-در-کمیسیون-بهداشت',
 'https://www.isna.ir/news/99080301248/بیمارستان-شهدای-هسته-ای-بوشهر-در-مراحل-پایانی-تکمیل-است']